# Pipeline Bronze to Silver - Processamento de Dados

## Validando a SparkSession

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

## Configurações iniciais - Azure ADLS Gen2

In [0]:
storageAccountName = "datalake7eadf73a479de9f7"
sasToken = "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-06-16T03:33:35Z&st=2025-06-15T19:33:35Z&spr=https&sig=fBJfCWK99G%2FGTBY81Y8eNQYEpOxgrzjVSaavlAB0np4%3D"

def mount_adls(blobContainerName):
    try:
        dbutils.fs.mount(
            source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
            mount_point = f"/mnt/{storageAccountName}/{blobContainerName}",
            extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
        )
        print(f"Container {blobContainerName} montado com sucesso!")
    except Exception as e:
        print(f"Falha ao montar {blobContainerName}: {e}")

## Montando containers necessários

In [0]:
mount_adls('bronze')
mount_adls('silver')

Falha ao montar bronze: An error occurred while calling o1915.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datalake7eadf73a479de9f7/bronze; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datalake7eadf73a479de9f7/bronze
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:135)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:69)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:1053)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.$anonfun$mount$1(DBUtilsCore.scala:1079)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:560)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:657)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags

## Verificando containers montados

In [0]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/mnt/datalake7eadf73a479de9f7/silver,wasbs://silver@datalake7eadf73a479de9f7.blob.core.windows.net,
/mnt/datalakefbca7dc4e981b9cb/landing-zone,wasbs://landing-zone@datalakefbca7dc4e981b9cb.blob.core.windows.net,
/databricks/mlflow-tracking,databricks/mlflow-tracking,sse-s3
/databricks-results,databricks-results,sse-s3
/mnt/datalakefbca7dc4e981b9cb/gold,wasbs://gold@datalakefbca7dc4e981b9cb.blob.core.windows.net,
/databricks/mlflow-registry,databricks/mlflow-registry,sse-s3
/mnt/datalake7eadf73a479de9f7/landing-zone,wasbs://landing-zone@datalake7eadf73a479de9f7.blob.core.windows.net,
/mnt/datalakefbca7dc4e981b9cb/silver,wasbs://silver@datalakefbca7dc4e981b9cb.blob.core.windows.net,
/mnt/datalakec8dc20f3798c1da9/bronze,wasbs://bronze@datalakec8dc20f3798c1da9.blob.core.windows.net,


## Visualizando arquivos na camada Bronze

In [0]:
display(dbutils.fs.ls(f"/mnt/{storageAccountName}/bronze"))

path,name,size,modificationTime
dbfs:/mnt/datalake7eadf73a479de9f7/bronze/achievement_unlocked/,achievement_unlocked/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/bronze/achievements/,achievements/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/bronze/developers/,developers/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/bronze/dlcs/,dlcs/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/bronze/game_genders/,game_genders/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/bronze/game_platforms/,game_platforms/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/bronze/game_tags/,game_tags/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/bronze/games/,games/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/bronze/genders/,genders/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/bronze/platforms/,platforms/,0,0


## Importando bibliotecas necessárias

In [0]:
from pyspark.sql.functions import current_timestamp, lit

## Função para padronização de nomes de colunas

In [0]:
def padronizar_nomes_colunas(df, nome_tabela):
    """
    Padroniza nomes de colunas seguindo convenções:
    - Converte para maiúscula
    - Substitui prefixos por nomes completos
    - Adiciona metadados de processamento
    """
    
    # Renomear colunas seguindo padrões
    for coluna in df.columns:
        novo_nome = coluna.upper()
        
        # Substituições de prefixos
        novo_nome = novo_nome.replace("CD_", "CODIGO_")
        novo_nome = novo_nome.replace("VL_", "VALOR_")
        novo_nome = novo_nome.replace("DT_", "DATA_")
        novo_nome = novo_nome.replace("NM_", "NOME_")
        novo_nome = novo_nome.replace("DS_", "DESCRICAO_")
        novo_nome = novo_nome.replace("NR_", "NUMERO_")
        novo_nome = novo_nome.replace("_UF", "_UNIDADE_FEDERATIVA")
        
        df = df.withColumnRenamed(coluna, novo_nome)
    
    # Remover colunas de metadados antigos se existirem
    colunas_remover = ["DATA_HORA_BRONZE", "NOME_ARQUIVO"]
    for col in colunas_remover:
        if col in df.columns:
            df = df.drop(col)
    
    # Adicionar metadados da camada Silver
    df = df.withColumn("NOME_ARQUIVO_BRONZE", lit(nome_tabela))
    df = df.withColumn("DATA_HORA_SILVER", current_timestamp())
    
    return df

## Processamento Bronze to Silver - Versão Massiva

In [0]:
def process_bronze_to_silver():
    """Processa todas as tabelas da camada Bronze para Silver"""
    
    bronze_path = f"/mnt/{storageAccountName}/bronze"
    silver_path = f"/mnt/{storageAccountName}/silver"
    
    tabelas_processadas = {}
    
    try:
        bronze_dirs = dbutils.fs.ls(bronze_path)
        
        for dir_info in bronze_dirs:
            if dir_info.isDir():
                table_name = dir_info.name.rstrip('/')
                print(f"\n=== Processando tabela: {table_name} ===")
                
                # Carregar dados da camada Bronze
                df_bronze = spark.read.format('delta').load(dir_info.path)
                print(f"Registros carregados: {df_bronze.count()}")
                
                # Aplicar transformações para Silver
                df_silver = padronizar_nomes_colunas(df_bronze, table_name)
                
                # Salvar na camada Silver
                df_silver.write.format('delta').mode('overwrite').save(f"{silver_path}/{table_name}")
                print(f"Tabela {table_name} salva na camada Silver")
                
                # Armazenar para validação
                tabelas_processadas[table_name] = df_silver
                
                # Mostrar esquema da tabela processada
                print(f"Esquema da tabela {table_name}:")
                df_silver.printSchema()
                
    except Exception as e:
        print(f"Erro no processamento: {str(e)}")
    
    return tabelas_processadas

## Executando o processamento Bronze to Silver

In [0]:
silver_tables = process_bronze_to_silver()


=== Processando tabela: achievement_unlocked ===
Registros carregados: 2
Tabela achievement_unlocked salva na camada Silver
Esquema da tabela achievement_unlocked:
root
 |-- ID: integer (nullable = true)
 |-- USER_ID: integer (nullable = true)
 |-- ACHIEVEMENT_ID: integer (nullable = true)
 |-- NOME_ARQUIVO_BRONZE: string (nullable = false)
 |-- DATA_HORA_SILVER: timestamp (nullable = false)


=== Processando tabela: achievements ===
Registros carregados: 2
Tabela achievements salva na camada Silver
Esquema da tabela achievements:
root
 |-- ID: integer (nullable = true)
 |-- GAME_ID: integer (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- NOME_ARQUIVO_BRONZE: string (nullable = false)
 |-- DATA_HORA_SILVER: timestamp (nullable = false)


=== Processando tabela: developers ===
Registros carregados: 2
Tabela developers salva na camada Silver
Esquema da tabela developers:
root
 |-- ID: integer (nullable = true)
 |-- NAME: string (nul

## Validação dos dados processados na camada Silver

In [0]:
print("=== VALIDAÇÃO CAMADA SILVER ===")
display(dbutils.fs.ls(f"/mnt/{storageAccountName}/silver"))

=== VALIDAÇÃO CAMADA SILVER ===


path,name,size,modificationTime
dbfs:/mnt/datalake7eadf73a479de9f7/silver/achievement_unlocked/,achievement_unlocked/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/silver/achievements/,achievements/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/silver/developers/,developers/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/silver/dlcs/,dlcs/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/silver/game_genders/,game_genders/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/silver/game_platforms/,game_platforms/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/silver/game_tags/,game_tags/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/silver/games/,games/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/silver/genders/,genders/,0,0
dbfs:/mnt/datalake7eadf73a479de9f7/silver/platforms/,platforms/,0,0


## Função para validar tabelas Silver

In [0]:
def validar_tabelas_silver():
    """Valida as tabelas criadas na camada Silver"""
    
    silver_path = f"/mnt/{storageAccountName}/silver"
    
    try:
        silver_dirs = dbutils.fs.ls(silver_path)
        
        for dir_info in silver_dirs:
            if dir_info.isDir():
                table_name = dir_info.name.rstrip('/')
                print(f"\n=== Validação: {table_name} ===")
                
                df = spark.read.format('delta').load(dir_info.path)
                print(f"Total de registros: {df.count()}")
                print(f"Colunas: {df.columns}")
                
                # Mostrar primeiras linhas
                print(f"Primeiras 5 linhas da tabela {table_name}:")
                df.limit(5).display()
                
    except Exception as e:
        print(f"Erro na validação: {str(e)}")

## Executando validação das tabelas Silver

In [0]:
validar_tabelas_silver()


=== Validação: achievement_unlocked ===
Total de registros: 2
Colunas: ['ID', 'USER_ID', 'ACHIEVEMENT_ID', 'NOME_ARQUIVO_BRONZE', 'DATA_HORA_SILVER']
Primeiras 5 linhas da tabela achievement_unlocked:


ID,USER_ID,ACHIEVEMENT_ID,NOME_ARQUIVO_BRONZE,DATA_HORA_SILVER
1,1,1,achievement_unlocked,2025-06-15T21:31:28.960+0000
2,2,2,achievement_unlocked,2025-06-15T21:31:28.960+0000



=== Validação: achievements ===
Total de registros: 2
Colunas: ['ID', 'GAME_ID', 'TITLE', 'DESCRIPTION', 'NOME_ARQUIVO_BRONZE', 'DATA_HORA_SILVER']
Primeiras 5 linhas da tabela achievements:


ID,GAME_ID,TITLE,DESCRIPTION,NOME_ARQUIVO_BRONZE,DATA_HORA_SILVER
1,1,Dragon Slayer,Defeat a dragon,achievements,2025-06-15T21:31:36.389+0000
2,2,Escape Hell,Complete the game,achievements,2025-06-15T21:31:36.389+0000



=== Validação: developers ===
Total de registros: 2
Colunas: ['ID', 'NAME', 'NOME_ARQUIVO_BRONZE', 'DATA_HORA_SILVER']
Primeiras 5 linhas da tabela developers:


...

ID,USER_ID,GAME_ID,RATING,COMMENT,NOME_ARQUIVO_BRONZE,DATA_HORA_SILVER
1,1,1,9,Amazing RPG!,reviews,2025-06-15T21:32:43.042+0000
2,2,2,8,Very fun gameplay.,reviews,2025-06-15T21:32:43.042+0000



=== Validação: tags ===
Total de registros: 2
Colunas: ['ID', 'NAME', 'NOME_ARQUIVO_BRONZE', 'DATA_HORA_SILVER']
Primeiras 5 linhas da tabela tags:


ID,NAME,NOME_ARQUIVO_BRONZE,DATA_HORA_SILVER
1,Singleplayer,tags,2025-06-15T21:32:49.234+0000
2,Indie,tags,2025-06-15T21:32:49.234+0000



=== Validação: users ===
Total de registros: 2
Colunas: ['ID', 'NAME', 'EMAIL', 'NOME_ARQUIVO_BRONZE', 'DATA_HORA_SILVER']
Primeiras 5 linhas da tabela users:


ID,NAME,EMAIL,NOME_ARQUIVO_BRONZE,DATA_HORA_SILVER
1,Alice,alice@example.com,users,2025-06-15T21:32:56.212+0000
2,Bob,bob@example.com,users,2025-06-15T21:32:56.212+0000


## Criação de tabelas Delta gerenciadas na camada Silver

In [0]:
def create_silver_managed_tables():
    """Cria tabelas Delta gerenciadas para a camada Silver"""
    
    database_name = "pipeline_silver"
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    
    silver_path = f"/mnt/{storageAccountName}/silver"
    
    try:
        silver_dirs = dbutils.fs.ls(silver_path)
        
        for dir_info in silver_dirs:
            if dir_info.isDir():
                table_name = dir_info.name.rstrip('/')
                
                # Ler dados da camada Silver
                df = spark.read.format('delta').load(dir_info.path)
                
                # Criar tabela gerenciada
                df.write.format('delta').mode('overwrite').saveAsTable(f"{database_name}.{table_name}")
                print(f"Tabela gerenciada criada: {database_name}.{table_name}")
                
    except Exception as e:
        print(f"Erro ao criar tabelas gerenciadas: {str(e)}")

## Executando criação de tabelas gerenciadas

In [0]:
create_silver_managed_tables()

Tabela gerenciada criada: seguro_silver.achievement_unlocked
Tabela gerenciada criada: seguro_silver.achievements
Tabela gerenciada criada: seguro_silver.developers
Tabela gerenciada criada: seguro_silver.dlcs
Tabela gerenciada criada: seguro_silver.game_genders
Tabela gerenciada criada: seguro_silver.game_platforms
Tabela gerenciada criada: seguro_silver.game_tags
Tabela gerenciada criada: seguro_silver.games
Tabela gerenciada criada: seguro_silver.genders
Tabela gerenciada criada: seguro_silver.platforms
Tabela gerenciada criada: seguro_silver.purchases
Tabela gerenciada criada: seguro_silver.reviews
Tabela gerenciada criada: seguro_silver.tags
Tabela gerenciada criada: seguro_silver.users


## Verificação das tabelas Silver gerenciadas

In [0]:
print("=== TABELAS SILVER GERENCIADAS ===")
spark.sql("SHOW TABLES IN pipeline_silver").show()

=== TABELAS SILVER GERENCIADAS ===
+-------------+--------------------+-----------+
|     database|           tableName|isTemporary|
+-------------+--------------------+-----------+
|seguro_silver|achievement_unlocked|      false|
|seguro_silver|        achievements|      false|
|seguro_silver|          developers|      false|
|seguro_silver|                dlcs|      false|
|seguro_silver|        game_genders|      false|
|seguro_silver|      game_platforms|      false|
|seguro_silver|           game_tags|      false|
|seguro_silver|               games|      false|
|seguro_silver|             genders|      false|
|seguro_silver|           platforms|      false|
|seguro_silver|           purchases|      false|
|seguro_silver|             reviews|      false|
|seguro_silver|                tags|      false|
|seguro_silver|               users|      false|
+-------------+--------------------+-----------+



## Função para análise de qualidade dos dados

In [0]:
def analise_qualidade_silver():
    """Realiza análise básica de qualidade dos dados na camada Silver"""
    
    database_name = "pipeline_silver"
    
    try:
        # Listar todas as tabelas
        tabelas = spark.sql(f"SHOW TABLES IN {database_name}").collect()
        
        for tabela in tabelas:
            nome_tabela = tabela['tableName']
            print(f"\n=== Análise de Qualidade: {nome_tabela} ===")
            
            # Contagem total de registros
            total_registros = spark.sql(f"SELECT COUNT(*) as total FROM {database_name}.{nome_tabela}").collect()[0]['total']
            print(f"Total de registros: {total_registros}")
            
            # Verificar valores nulos por coluna
            df = spark.sql(f"SELECT * FROM {database_name}.{nome_tabela}")
            
            print("Valores nulos por coluna:")
            for coluna in df.columns:
                if coluna not in ['NOME_ARQUIVO_BRONZE', 'DATA_HORA_SILVER']:
                    nulos = df.filter(df[coluna].isNull()).count()
                    percentual = (nulos / total_registros) * 100 if total_registros > 0 else 0
                    print(f"  {coluna}: {nulos} ({percentual:.2f}%)")
            
    except Exception as e:
        print(f"Erro na análise de qualidade: {str(e)}")

## Executando análise de qualidade

In [0]:
analise_qualidade_silver()


=== Análise de Qualidade: achievement_unlocked ===
Total de registros: 2
Valores nulos por coluna:
  ID: 0 (0.00%)
  USER_ID: 0 (0.00%)
  ACHIEVEMENT_ID: 0 (0.00%)

=== Análise de Qualidade: achievements ===
Total de registros: 2
Valores nulos por coluna:
  ID: 0 (0.00%)
  GAME_ID: 0 (0.00%)
  TITLE: 0 (0.00%)
  DESCRIPTION: 0 (0.00%)

=== Análise de Qualidade: developers ===
Total de registros: 2
Valores nulos por coluna:
  ID: 0 (0.00%)
  NAME: 0 (0.00%)

=== Análise de Qualidade: dlcs ===
Total de registros: 2
Valores nulos por coluna:
  ID: 0 (0.00%)
  GAME_ID: 0 (0.00%)
  TITLE: 0 (0.00%)

=== Análise de Qualidade: game_genders ===
Total de registros: 2
Valores nulos por coluna:
  ID: 0 (0.00%)
  GAME_ID: 0 (0.00%)
  GENDER_ID: 0 (0.00%)

=== Análise de Qualidade: game_platforms ===
Total de registros: 2
Valores nulos por coluna:
  ID: 0 (0.00%)
  GAME_ID: 0 (0.00%)
  PLATFORM_ID: 0 (0.00%)

=== Análise de Qualidade: game_tags ===
Total de registros: 2
Valores nulos por coluna:
 